In [1]:
import cv2
import numpy as np

#initialize the webcam
cap = cv2.VideoCapture(0)

#read the first frame as the background
ret, background = cap.read()

#convert the background to grayscale
background_gray = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)

#apply Gaussian blur to the background to reduce noise
background_blur = cv2.GaussianBlur(background_gray, (21, 21), 0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    #convert the current frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #apply Gaussian blur to the frame to reduce noise
    gray_blur = cv2.GaussianBlur(gray, (21, 21), 0)
    
    #compute the absolute difference between the background and the current frame
    diff = cv2.absdiff(background_blur, gray_blur)
    
    #apply a threshold to get the foreground mask
    _, mask = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
    
    #invert the mask to get the background mask
    mask_inv = cv2.bitwise_not(mask)
    
    #create a white background of the same size as the frame
    white_background = np.full_like(frame, 255)
    
    #use the mask to extract the moving object from the current frame
    foreground = cv2.bitwise_and(frame, frame, mask=mask)
    
    #use the inverse mask to extract the background from the white background
    white_bg = cv2.bitwise_and(white_background, white_background, mask=mask_inv)
    
    #combine the foreground (moving object) with the white background
    result = cv2.add(foreground, white_bg)
    
    #show the result
    cv2.imshow('Moving Object with White Background', result)
    
    #press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#release the webcam and close the windows
cap.release()
cv2.destroyAllWindows()